#The purpose of this notebook is to create a series of interest rate variables in which we calculate the 'relevant' interest rate in the market at that time as compared to how much time is left on the loan. For example, we shouldn't consider refinancing a 20 year loan at a 20 year rate if there's only 5 years left on the loan. Since there may be a relationship between the rate at which someone could refinance and their prepayment behavior we opted to include this. 

In [1]:
import pandas as pd
import numpy as np
import pickle
pd.options.display.max_columns = 1000

In [18]:
rates = pd.read_csv('D:/Bank Data/Historical Market Rates.csv')
america = pd.read_csv('D:/Bank Data/FRED Data/America_Data.csv')
quarterly_bank1 = pd.read_csv('D:/Darling Data/quarterly_banks1_nat_data.csv')
# qb4 = pickle.load(open('D:/Darling Data/quarterly_banks4.p', 'rb'))
all_banks = pickle.load(open('D:/Darling Data/All_Banks5.p', 'rb'))            
rates.head()

quarterly_bank1['Maturity_Date'] = pd.to_datetime(quarterly_bank1['Maturity_Date'])
quarterly_bank1['As_Of_Date'] = pd.to_datetime(quarterly_bank1['As_Of_Date'])
america['As_Of_Date'] = pd.to_datetime(america['As_Of_Date'])

rates.columns = ['As_Of_Date', '1 Month FHLB', '3 Month FHLB', '6 Month FHLB',
       '1 Year FHLB', '2 Year FHLB', '3 Year FHLB', '4 Year FHLB',
       '5 Year FHLB', '6 Year FHLB', '7 Year FHLB', '8 Year FHLB',
       '9 Year FHLB', '10 Year FHLB', '15 Year FHLB', '20 Year FHLB',
       '1m US T-Bill (BEY)', '3m US T-Bill (BEY)', '6m US T-Bill (BEY)',
       '1y US T-Bill', '2y US T-Note', '3y US T-Note', '5y US T-Note',
       '7y US T-Note', '10y US T-Note', '20y US T-Bond', '30y US T-Bond',
       '1 Month USD LIBOR', '3 Month USD LIBOR', '6 Month USD LIBOR',
       '12 Month USD LIBOR', '1YR LIBOR Swap', '2YR LIBOR Swap',
       '3YR LIBOR Swap', '4YR LIBOR Swap', '5YR LIBOR Swap', '6YR LIBOR Swap',
       '7YR LIBOR Swap', '8YR LIBOR Swap', '9YR LIBOR Swap', '10YR LIBOR Swap',
       '15YR LIBOR Swap', '20YR LIBOR Swap', '30YR LIBOR Swap', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56',
       'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60']
rates['As_Of_Date'] = pd.to_datetime(rates['As_Of_Date'])
# america.As_Of_Date


C:\Users\thomb\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2,38,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:

rates
libor = rates[['As_Of_Date', '1 Month USD LIBOR', '3 Month USD LIBOR', '6 Month USD LIBOR',
       '12 Month USD LIBOR', '1YR LIBOR Swap', '2YR LIBOR Swap',
       '3YR LIBOR Swap', '4YR LIBOR Swap', '5YR LIBOR Swap', '6YR LIBOR Swap',
       '7YR LIBOR Swap', '8YR LIBOR Swap', '9YR LIBOR Swap', '10YR LIBOR Swap',
       '15YR LIBOR Swap', '20YR LIBOR Swap', '30YR LIBOR Swap']]

In [20]:
dates=america[['As_Of_Date']]
dates['As_Of_Date'] = pd.to_datetime(dates['As_Of_Date'])
libor['As_Of_Date'] = pd.to_datetime(libor['As_Of_Date'])


libor = pd.merge(dates, libor, on = 'As_Of_Date', how = 'left')

C:\Users\thomb\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\thomb\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
#Filling in all missing dates with the rate above. Basically filling in weekends and holidays with the prior biz day. 
libor = libor.fillna(method='ffill')

In [22]:
#Ensuring columns are datetimes. 
qtr_date = quarterly_bank1.As_Of_Date.unique()
qtr_date=pd.DataFrame(qtr_date)
qtr_date.columns = ['As_Of_Date']
qtr_date['As_Of_Date'] = pd.to_datetime(qtr_date['As_Of_Date'])
libor['As_Of_Date'] = pd.to_datetime(libor['As_Of_Date'])

In [23]:
#Merging the dataframes to retain only the relevant quarters. 
libor_qtr = pd.merge(qtr_date, libor, on='As_Of_Date', how = 'left' )

# libor_qtr['1 Month USD LIBOR']=np.round(libor_qtr['1 Month USD LIBOR'], 2)


# for i in libor_qtr.columns[1:]:
#     libor_qtr[str(i)] = np.round(libor_qtr[str(i)], 2)
#     libor_qtr[str(i)] = libor_qtr[str(i)].astype(float)
# libor_qtr['As_Of_Date'] = pd.to_datetime(libor_qtr['As_Of_Date'])

In [24]:
libor_qtr

,As_Of_Date,1 Month USD LIBOR,3 Month USD LIBOR,6 Month USD LIBOR,12 Month USD LIBOR,1YR LIBOR Swap,2YR LIBOR Swap,3YR LIBOR Swap,4YR LIBOR Swap,5YR LIBOR Swap,6YR LIBOR Swap,7YR LIBOR Swap,8YR LIBOR Swap,9YR LIBOR Swap,10YR LIBOR Swap,15YR LIBOR Swap,20YR LIBOR Swap,30YR LIBOR Swap
0,2003-12-31,1.12000,1.15000,1.22000,1.46000,1.46,2.15,2.76,3.26,3.67,3.95,4.17,4.35,4.52,4.65,5.09,5.30,5.39
1,2004-03-31,1.09000,1.11000,1.16000,1.35000,1.34,1.88,2.39,2.82,3.17,3.47,3.71,3.91,4.08,4.23,4.74,4.97,5.09
2,2004-06-30,1.37000,1.61000,1.94000,2.46000,2.38,3.09,3.59,3.97,4.26,4.49,4.67,4.83,4.95,5.06,5.44,5.61,5.67
3,2004-09-30,1.84000,2.02000,2.20000,2.48000,2.49,2.94,3.29,3.58,3.81,4.00,4.17,4.32,4.45,4.56,4.96,5.15,5.23
4,2004-12-31,2.40000,2.56000,2.78000,3.10000,3.10,3.45,3.66,3.85,4.03,4.18,4.31,4.44,4.54,4.64,4.98,5.14,5.23
5,2005-03-31,2.87000,3.12000,3.40000,3.85000,3.81,4.19,4.39,4.52,4.62,4.70,4.77,4.83,4.89,4.94,5.12,5.21,5.26
6,2005-06-30,3.34000,3.52000,3.71000,3.88000,3.91,3.98,4.04,4.08,4.13,4.17,4.21,4.25,4.29,4.33,4.50,4.58,4.63
7,2005-09-30,3.86000,4.07000,4.23000,4.44000,4.49,4.57,4.60,4.63,4.65,4.68,4.71,4.73,4.76,4.79,4.92,4.99,5.03
8,2005-12-31,4.39000,4.54000,4.70000,4.84000,4.85,4.85,4.84,4.86,4.88,4.88,4.89,4.91,4.92,4.94,5.02,5.06,5.07
9,2006-03-31,4.83000,5.00000,5.14000,5.29000,5.29,5.28,5.27,5.28,5.30,5.33,5.34,5.36,5.37,5.39,5.45,5.48,5.49


In [25]:
#Extracting the interest rates we'll be using in the rate dictionary and renaming them for quicker extraction. 
quarter_rates = libor_qtr[['As_Of_Date', '6 Month USD LIBOR', '1YR LIBOR Swap',
       '2YR LIBOR Swap', '3YR LIBOR Swap', '4YR LIBOR Swap', '5YR LIBOR Swap',
       '6YR LIBOR Swap', '7YR LIBOR Swap', '8YR LIBOR Swap', '9YR LIBOR Swap',
       '10YR LIBOR Swap', '15YR LIBOR Swap', '20YR LIBOR Swap',
       '30YR LIBOR Swap']]

quarter_rates.columns = ['As_Of_Date', '0', '1',
       '2', '3', '4', '5',
       '6', '7', '8', '9',
       '10', '15', '20',
       '30']

quarter_rates

,As_Of_Date,0,1,2,3,4,5,6,7,8,9,10,15,20,30
0,2003-12-31,1.22000,1.46,2.15,2.76,3.26,3.67,3.95,4.17,4.35,4.52,4.65,5.09,5.30,5.39
1,2004-03-31,1.16000,1.34,1.88,2.39,2.82,3.17,3.47,3.71,3.91,4.08,4.23,4.74,4.97,5.09
2,2004-06-30,1.94000,2.38,3.09,3.59,3.97,4.26,4.49,4.67,4.83,4.95,5.06,5.44,5.61,5.67
3,2004-09-30,2.20000,2.49,2.94,3.29,3.58,3.81,4.00,4.17,4.32,4.45,4.56,4.96,5.15,5.23
4,2004-12-31,2.78000,3.10,3.45,3.66,3.85,4.03,4.18,4.31,4.44,4.54,4.64,4.98,5.14,5.23
5,2005-03-31,3.40000,3.81,4.19,4.39,4.52,4.62,4.70,4.77,4.83,4.89,4.94,5.12,5.21,5.26
6,2005-06-30,3.71000,3.91,3.98,4.04,4.08,4.13,4.17,4.21,4.25,4.29,4.33,4.50,4.58,4.63
7,2005-09-30,4.23000,4.49,4.57,4.60,4.63,4.65,4.68,4.71,4.73,4.76,4.79,4.92,4.99,5.03
8,2005-12-31,4.70000,4.85,4.85,4.84,4.86,4.88,4.88,4.89,4.91,4.92,4.94,5.02,5.06,5.07
9,2006-03-31,5.14000,5.29,5.28,5.27,5.28,5.30,5.33,5.34,5.36,5.37,5.39,5.45,5.48,5.49


In [ ]:
#Dataframe with just libor swaps for years 1 through 30 for all quarters. 
libor_dict_2 = {}
quarter_rates['As_Of_Date'] = pd.to_datetime(quarter_rates['As_Of_Date'])

for i, j in enumerate(quarter_rates.columns[1:]):
    libor_dict_2[j] = {}
for m,n in enumerate(libor_dict_2.keys()):
    for x,y in enumerate(quarter_rates['As_Of_Date']):
        libor_dict_2[n][y] = quarter_rates[n][x]
libor_dict_2

In [27]:
print(libor_dict_2.keys())
print('_____________________________________________________________________\n')
print(quarter_rates.columns)
print('_____________________________________________________________________\n')
print(quarterly_bank1.columns)

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '15', '20', '30'])
_____________________________________________________________________

Index(['As_Of_Date', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '15', '20', '30'],
      dtype='object')
_____________________________________________________________________

Index(['Bank', 'As_Of_Date', 'Account_Number', 'PrepayClassCode',
       'Fixed/Adjustable', 'Term_Structure', 'Original_Term_Range',
       'Original_Term', 'Loan_Age_Range', 'Current_Interest_Rate',
       'Interest_Margin', 'Lifetime_Rate_Cap', 'Lifetime_Rate_Floor',
       'Interest_Rate_Change_Frequency', 'Maturity_Date', 'Quarters_Remaining',
       'Reprice_Term', 'Next_Repricing_Date', 'Actual_Payment',
       'Expected_Payment', 'Gross_Prepayment', 'Missed_Payment',
       'P_&_I_Payment', 'Next_Payment_Date', 'Curtailment', 'Draws',
       'Total_Prepayment', 'Modification', 'Refinance', 'PayOff', 'CPR',
       'Prior_Quarter_Bal

In [28]:

quarterly_bank1.Quarters_Remaining

quarterly_bank1['Relevant_Rate'] = np.nan

In [29]:
quarterly_bank1.columns

Index(['Bank', 'As_Of_Date', 'Account_Number', 'PrepayClassCode',
       'Fixed/Adjustable', 'Term_Structure', 'Original_Term_Range',
       'Original_Term', 'Loan_Age_Range', 'Current_Interest_Rate',
       'Interest_Margin', 'Lifetime_Rate_Cap', 'Lifetime_Rate_Floor',
       'Interest_Rate_Change_Frequency', 'Maturity_Date', 'Quarters_Remaining',
       'Reprice_Term', 'Next_Repricing_Date', 'Actual_Payment',
       'Expected_Payment', 'Gross_Prepayment', 'Missed_Payment',
       'P_&_I_Payment', 'Next_Payment_Date', 'Curtailment', 'Draws',
       'Total_Prepayment', 'Modification', 'Refinance', 'PayOff', 'CPR',
       'Prior_Quarter_Balance', 'Balance_Owned_Participation',
       'Classification_Code', 'Repricing_Code', 'Payment_Frequency_Code',
       'Current_Index_Code', 'Type_Code', 'Class_Code', 'Collateral_Code',
       'Interest/Principal_Only_Code', 'Repricing_Frequency_Code', 'LOC_Code',
       'Multi_Family_Code', 'Custom_Tag_1', 'Custom_Tag_2', 'Custom_Tag_3',
       'Pri

In [30]:
# df['J3'] = df.apply(lambda row:lst[row['J1']:row['J2']],axis=1)

# def relevant_rate(row):
    
#     try:
#         x = row['Quarters_Remaining']//4
#         if x>0 and x<11:
#             row['Relevant_Rate'] = np.round(libor_dict_2[str(x)][row['As_Of_Date'][i]],2)
#         elif x>10 and x<16:
#             row['Relevant_Rate'][i] = np.round(libor_dict_2['15'][row['As_Of_Date'][i]],2)
#         elif x>15 and x<21:
#             row['Relevant_Rate'][i] = np.round(libor_dict_2['20'][row['As_Of_Date'][i]],2)
#         elif x>20:
#             row['Relevant_Rate'][i] = np.round(libor_dict_2['30'][row['As_Of_Date'][i]],2)
#         elif x<=0:
#             row['Relevant_Rate'][i] = 0
#     except:
#         continue

In [31]:
#This is the function to apply the 'relevant rate' to each quarter. The relevant rate is the interest rate benchmark
#for the time to maturity for the loan. Requires dictionary with the rates and dates. 

def relevant_rate(row):

    x = row['Quarters_Remaining']//4
    if 0 <= x < 11:
        return libor_dict_2[str(int(x))][pd.to_datetime(row['As_Of_Date'])]
    elif 10 < x < 16:
        return libor_dict_2['15'][pd.to_datetime(row['As_Of_Date'])]
    elif 15 < x < 21:
        return libor_dict_2['20'][pd.to_datetime(row['As_Of_Date'])]
    elif x > 20:
        return libor_dict_2['30'][pd.to_datetime(row['As_Of_Date'])]
    elif x < 0:
        return 0
    else:
        return np.nan

In [32]:
#Testing extraction from the dictionary. 
libor_dict_2[str(5//4)][pd.to_datetime('2003-12-31')]

1.46

In [33]:
#Testing .apply()
def test_funk(row):
    return pd.to_datetime(row['As_Of_Date'])


test = pd.DataFrame(quarterly_bank1.iloc[0:2,:])
test
test.apply(test_funk, axis = 1)

0   2003-12-31
1   2003-12-31
dtype: datetime64[ns]

In [34]:
#Testing. 
test['Test'] = test.apply(relevant_rate, axis = 1)
test['Test']

0    5.39
1    5.09
Name: Test, dtype: float64

In [19]:
#Apply the function. 
# quarterly_bank1['Relevant_Rate'] = quarterly_bank1.apply(relevant_rate, axis = 1)

In [48]:
#Correct the missing maturity dates to improve quarters remaining and relevant rate. 
#Only filling in the situation where the last value is missing.
onenullmissmat = all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()[all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()==1]
onenullmatlist = list(onenullmissmat.index)
miss_mat_one = all_banks[all_banks.Account_Number.isin(onenullmatlist)]
miss_mat_one_group = miss_mat_one.groupby('Account_Number')


In [49]:
miss_mat_one.sort_values('Account_Number')

,Bank,As_Of_Date,Account_Number,PrepayClassCode,Fixed/Adjustable,Term_Structure,Original_Term_Range,Original_Term,Loan_Age_Range,Current_Interest_Rate,Interest_Margin,Lifetime_Rate_Cap,Lifetime_Rate_Floor,Interest_Rate_Change_Frequency,Maturity_Date,NewLoan,Reprice_Term,Next_Repricing_Date,Actual_Payment,Expected_Payment,Gross_Prepayment,Missed_Payment,P_&_I_Payment,Next_Payment_Date,Curtailment,Draws,Total_Prepayment,Modification,Refinance,PayOff,CPR,Prior_Quarter_Balance,Balance_Owned_Participation,Classification_Code,Repricing_Code,Payment_Frequency_Code,Current_Index_Code,Type_Code,Class_Code,Collateral_Code,Interest/Principal_Only_Code,Repricing_Frequency_Code,LOC_Code,Multi_Family_Code,Custom_Tag_1,Custom_Tag_2,Custom_Tag_3,Prior_Interest_Rate,Prior_Maturity_Date,Prior_Origination_Date,Prior_Payment_Fequency,Prior_Next_Reprice_Date,Prior_Internal_Credit_Rating,Prior_Branch,Prior_LOC_Code,Originating_Branch,Internal_Credit_Rating,Final_Payment


In [46]:
counter = 1
for index, group in miss_mat_one_group:
    print(index)
    index = group.index
    all_banks['Maturity_Date'][index] = group.sort_values('As_Of_Date').Maturity_Date.fillna(method='ffill')

1082551


C:\Users\thomb\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


7200009669
7200097312
7200110843
7200111303
7200114163
9000035576


In [21]:
onenullmissmatcheck=all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()[all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()==1]
len(onenullmissmatcheck)

0

In [36]:
all_banks['Maturity_Date'] = pd.to_datetime(all_banks['Maturity_Date'])


#all_banks is the new dataset to use. 
def diff_month(row):
    return ((row['Maturity_Date'].year - row['As_Of_Date'].year) * 12 + (row['Maturity_Date'].month - row['As_Of_Date'].month))//3
all_banks['Quarters_Remaining'] = all_banks.apply(diff_month, axis=1)

In [51]:
all_banks.info()
# all_banks['Maturity_Date'] = pd.to_datetime(all_banks['Maturity_Date'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654337 entries, 0 to 654336
Data columns (total 59 columns):
Bank                              654337 non-null int64
As_Of_Date                        654337 non-null datetime64[ns]
Account_Number                    654337 non-null object
PrepayClassCode                   654337 non-null object
Fixed/Adjustable                  654337 non-null object
Term_Structure                    631281 non-null object
Original_Term_Range               654337 non-null object
Original_Term                     654314 non-null float64
Loan_Age_Range                    654337 non-null object
Current_Interest_Rate             654337 non-null float64
Interest_Margin                   654337 non-null float64
Lifetime_Rate_Cap                 654337 non-null float64
Lifetime_Rate_Floor               654337 non-null float64
Interest_Rate_Change_Frequency    654337 non-null float64
Maturity_Date                     649847 non-null datetime64[ns]
NewLoan      

In [37]:
#apply function to the all_ banks.
all_banks['Relevant_Rate'] = all_banks.apply(relevant_rate, axis=1)

In [38]:
all_banks['Relevant_Rate'].isnull().value_counts()
all_banks['Maturity_Date'].isnull().value_counts()

False    649840
True       4497
Name: Maturity_Date, dtype: int64

In [39]:
#Reviewing Results.
print(all_banks['Relevant_Rate'])
print(all_banks['Quarters_Remaining'])
# print(quarterly_bank1['As_Of_Date'][416097])
all_banks['Relevant_Rate'].isnull().value_counts()
all_banks['Maturity_Date'].isnull().value_counts()

all_banks['Relevant_Rate_Diff'] = all_banks['Current_Interest_Rate']-all_banks['Relevant_Rate']
#416107

0          NaN
1         5.09
2         5.09
3         5.30
4         5.09
5         5.09
6          NaN
7          NaN
8         5.39
9         5.39
10        5.09
11        5.09
12        5.39
13         NaN
14        5.39
15         NaN
16        5.09
17        5.09
18        5.39
19        5.09
20         NaN
21        5.39
22        4.65
23         NaN
24         NaN
25        5.09
26         NaN
27        5.09
28         NaN
29        5.39
          ... 
654307    2.43
654308    1.94
654309    1.57
654310    2.08
654311    2.08
654312    2.43
654313    1.57
654314    1.57
654315    1.57
654316    2.49
654317    1.57
654318    2.14
654319    1.57
654320    2.43
654321    1.94
654322    1.94
654323    2.43
654324    1.57
654325    1.74
654326    1.51
654327    1.94
654328    1.94
654329    1.51
654330    1.51
654331    2.19
654332    1.57
654333    2.08
654334    2.29
654335    1.57
654336    2.49
Name: Relevant_Rate, Length: 654337, dtype: float64
0           NaN
1          58.0
2

In [40]:
all_banks.columns
america.columns
all_banks1 = pd.merge(all_banks, america, on = 'As_Of_Date', how = 'left')

In [59]:
len(america.As_Of_Date)

5585

In [42]:
#Find the accounts with missing account numbers. 
missmaturity = all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.unique()
missmaturity_df=all_banks[all_banks['Maturity_Date'].isnull()==True]
missmaturity_df.Bank.value_counts()
missmaturity_df.Maturity_Date.unique()
miss_mat_remove = missmaturity_df.Account_Number.unique()
# onenullmissmat=all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()[all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()==1]

In [53]:
onenullmissmat=all_banks1[all_banks1['Maturity_Date'].isnull()==True].Account_Number.value_counts()[all_banks1[all_banks1['Maturity_Date'].isnull()==True].Account_Number.value_counts()==1]

In [43]:
missmaturity_df
pickle.dump(missmaturity_df ,open('D:/Bank Data/all_banks5_missing_maturity_dates.p', 'wb'))

In [44]:
miss_mat_remove
all_banks1=all_banks1[~all_banks1.Account_Number.isin(miss_mat_remove)]

In [55]:
#Dump to pickle. 
# pickle.dump(all_banks1 ,open('D:/Bank Data/all_banks5_nat.p', 'wb'))

In [57]:
all_banks1.shape

(649729, 80)

In [45]:
missmaturity_df[missmaturity_df.Account_Number=='5700087389'].sort_values('As_Of_Date')

,Bank,As_Of_Date,Account_Number,PrepayClassCode,Fixed/Adjustable,Term_Structure,Original_Term_Range,Original_Term,Loan_Age_Range,Current_Interest_Rate,Interest_Margin,Lifetime_Rate_Cap,Lifetime_Rate_Floor,Interest_Rate_Change_Frequency,Maturity_Date,NewLoan,Reprice_Term,Next_Repricing_Date,Actual_Payment,Expected_Payment,Gross_Prepayment,Missed_Payment,P_&_I_Payment,Next_Payment_Date,Curtailment,Draws,Total_Prepayment,Modification,Refinance,PayOff,CPR,Prior_Quarter_Balance,Balance_Owned_Participation,Classification_Code,Repricing_Code,Payment_Frequency_Code,Current_Index_Code,Type_Code,Class_Code,Collateral_Code,Interest/Principal_Only_Code,Repricing_Frequency_Code,LOC_Code,Multi_Family_Code,Custom_Tag_1,Custom_Tag_2,Custom_Tag_3,Prior_Interest_Rate,Prior_Maturity_Date,Prior_Origination_Date,Prior_Payment_Fequency,Prior_Next_Reprice_Date,Prior_Internal_Credit_Rating,Prior_Branch,Prior_LOC_Code,Originating_Branch,Internal_Credit_Rating,Final_Payment,Quarters_Remaining,Relevant_Rate,Relevant_Rate_Diff
255,4,2003-12-31,5700087389,C & I,Floating,1M,<12,0.0,61-120,5.50,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2004-01-31 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,102266.51,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369,NaN,0,NaN,NaN,NaN
785,4,2004-03-31,5700087389,C & I,Floating,1M,<12,0.0,61-120,5.50,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2004-04-30 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,102266.51,93161.88,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,5.50,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
1696,4,2004-06-30,5700087389,C & I,Floating,1M,<12,0.0,61-120,5.50,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2004-07-31 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93161.88,83574.31,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,5.50,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
2239,4,2004-09-30,5700087389,C & I,Floating,1M,<12,0.0,61-120,6.25,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2004-10-31 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83574.31,73574.31,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,5.50,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
2683,4,2004-12-31,5700087389,C & I,Floating,1M,<12,0.0,61-120,6.75,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2005-01-31 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73574.31,65104.83,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,6.25,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
3686,4,2005-03-31,5700087389,C & I,Floating,1M,<12,0.0,61-120,7.25,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2005-04-30 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65104.83,55104.83,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,6.75,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
3754,4,2005-06-30,5700087389,C & I,Floating,1M,<12,0.0,61-120,7.50,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2005-07-31 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55104.83,50104.83,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,7.25,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
4708,4,2005-09-30,5700087389,C & I,Floating,1M,<12,0.0,61-120,8.25,1.5,0.0,0.0,0.0,NaT,False,<6,NaN,0.0,0.0,0.0,0.0,0.0,2005-10-31 00:00:00.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50104.83,45104.83,C & I,Floating,Monthly,LR15,001-401,001-401,NONE,Interest Only,1M,N,N,Non-SpringStone,Non-Sm Business,Non-Auto,7.50,NaN,1997-05-28 00:00:00.000,1,NaN,NaN,369,N,369,NaN,0,NaN,NaN,NaN
5654,4,2005-12-31,5700087389,C

In [54]:
len(onenullmissmat)
# onenullmatlist=list(onenullmissmat.index)

0

In [51]:
miss_mat_all = all_banks[all_banks.Account_Number.isin(missmaturity)]
miss_mat_one = all_banks[all_banks.Account_Number.isin(onenullmatlist)]

In [52]:
print(len(missmaturity))
print('All',miss_mat_all.shape)
print('One', miss_mat_one.shape)

miss_mat_all.sort_values('Account_Number')

miss_mat_group=miss_mat_all.groupby('Account_Number')

miss_mat_one_group = miss_mat_one.groupby('Account_Number')

1059
All (4608, 61)
One (52, 61)


In [61]:
#practicing filling in the data.
counter = 1
for index, group in miss_mat_one_group:
    print(index)
    index = group.index
    all_banks['Maturity_Date'][index] = group.sort_values('As_Of_Date').Maturity_Date.fillna(method='ffill')


In [67]:
#Whats going on with the two index values on all_banks?
index = X.index
index
all_banks['Maturity_Date'][index]=X.sort_values('As_Of_Date').Maturity_Date.fillna(method='ffill')

C:\Users\thomb\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# all_banks.iloc[index,:]
# all_banks['Original_Index']=all_banks['index']
# all_banks['Original_Index']
# all_banks.drop(all_banks.columns[0])

In [ ]:
#print out all of the relevant rates to verify. 

counter = 0
for index, group in miss_mat_group:
    counter+=1
    print(index)
    display(group.sort_values('As_Of_Date'))
    if counter>10:
        break

In [ ]:
#Correct the missing maturity dates to improve quarters remaining and relevant rate. 
#Only filling in the situation where the last value is missing.


# onenullmissmat = all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()[all_banks[all_banks['Maturity_Date'].isnull()==True].Account_Number.value_counts()==1]
# onenullmatlist = list(onenullmissmat.index)
# miss_mat_one = all_banks[all_banks.Account_Number.isin(onenullmatlist)]
# miss_mat_one_group = miss_mat_one.groupby('Account_Number')

# counter = 1
# for index, group in miss_mat_one_group:
#     print(index)
#     index = group.index
#     all_banks['Maturity_Date'][index] = group.sort_values('As_Of_Date').Maturity_Date.fillna(method='ffill')

In [23]:
#Error handling. 
g = pd.DataFrame({'Quarters_Remaining': [5.0, 2.0, 42], 'As_Of_Date': ['2017-09-30','2017-09-30','2017-09-30']})
g.apply(relevant_rate, axis = 1)

0    1.57
1    1.51
2    2.29
dtype: float64

In [24]:
H = pd.DataFrame({'Test1': [5.0, 2.0, 42, 6], 'Test2': [3, 1, 40, np.nan]})
H

,Test1,Test2
0,5.0,3.0
1,2.0,1.0
2,42.0,40.0
3,6.0,NaN


In [25]:
H['Test1']-H['Test2']

0    2.0
1    1.0
2    2.0
3    NaN
dtype: float64

In [26]:
quarterly_bank1['Relevant_Rate_Diff'] = quarterly_bank1['Current_Interest_Rate']-quarterly_bank1['Relevant_Rate']

In [27]:
quarterly_bank1['Relevant_Rate_Diff']

0         0.410
1        -0.090
2         1.160
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12        8.410
13        8.410
14        3.550
15          NaN
16       -1.590
17          NaN
18          NaN
19        1.530
20        2.410
21          NaN
22       -0.090
23          NaN
24        8.410
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
          ...  
416090    2.406
416091    3.210
416092    3.930
416093    3.965
416094    1.597
416095    3.410
416096    0.970
416097    6.430
416098    3.180
416099    3.490
416100    3.430
416101    5.560
416102    3.810
416103    2.777
416104    4.250
416105    4.060
416106    1.760
416107    3.740
416108    3.250
416109    3.680
416110    1.970
416111    4.060
416112    3.250
416113    1.596
416114    3.760
416115    2.240
416116    4.680
416117    2.740
416118    1.046
416119    1.656
Name: Relevant_Rate_Diff

In [28]:
#Putting it all together.
#Function


# def relevant_rate(row):

#     x = row['Quarters_Remaining']//4
#     if 0 < x < 11:
#         return libor_dict_2[str(int(x))][pd.to_datetime(row['As_Of_Date'])]
#     elif 10 < x < 16:
#         return libor_dict_2['15'][pd.to_datetime(row['As_Of_Date'])]
#     elif 15 < x < 21:
#         return libor_dict_2['20'][pd.to_datetime(row['As_Of_Date'])]
#     elif x > 20:
#         return libor_dict_2['30'][pd.to_datetime(row['As_Of_Date'])]
#     elif x <= 0:
#         return 0
#     else:
#         return np.nan

# #Apply the function
# quarterly_bank1['Relevant_Rate'] = quarterly_bank1.apply(relevant_rate, axis = 1)

# #Make the third columns the differences. 
# quarterly_bank1['Relevant_Rate_Diff'] = quarterly_bank1['Current_Interest_Rate']-quarterly_bank1['Relevant_Rate']

In [29]:
#Checking the NaN values for the calculated fields.
quarterly_bank1.columns
quarterly_bank1.Relevant_Rate_Diff.isnull().value_counts()

False    412453
True       3667
Name: Relevant_Rate_Diff, dtype: int64

In [33]:
#Modifying the quarters left function for .apply()
def diff_month(row):
    return ((row['Maturity_Date'].year - row['As_Of_Date'].year) * 12 + (row['Maturity_Date'].month - row['As_Of_Date'].month))//3


In [61]:
#Write to pickle. 
#qb4.to_pickle('D:/Bank Data/quarterly_banks4_w_rates.p')